In [13]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-11 02:59:54--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-11 02:59:54 (64.4 MB/s) - ‘minsearch.py.2’ saved [4073/4073]



In [14]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [35]:
import minsearch

In [36]:
import json

In [41]:
from openai import OpenAI

In [37]:
with open('documents.json', 'rt') as f_in:
    doc_raw = json.load(f_in)

In [38]:
documents = []

for course_dict in doc_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [39]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
)

In [40]:
index.fit(documents)

In [42]:
client = OpenAI()

In [69]:
def search(query):
    boost = {'question': 3.0, 'section' : 0.5}

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [70]:
def build_prompt(query, search_results):
    prompt_template = """

    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION:{question}
    CONTEXT:{context}
    """.strip()
    
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context= context).strip()
    return prompt

In [71]:
def llm(prompt):
    response = client.chat.completions.create(
    model = 'gpt-4o',
    messages=[{"role":"user", "content":prompt}]
)
    return response.choices[0].message.content

In [72]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [73]:
rag(query)

'To run Kafka in your project directory using Java, you should execute the following command in your terminal:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'